# Import Libraries & Data

In [3]:
import os
import glob
import re
import numpy as np
import pandas as pd

### Load and connect all csv files

In [6]:
# file paths

kaggle_outputs = '/home/maria/Desktop/Deception_project/1. Youtube Extractions/kaggle csv outputs/'

In [7]:
# %time

# # load all csv openface outputs (from DOLOS)

# kaggle_files = glob.glob(os.path.join(kaggle_outputs, '*.csv'))
# df_list = [pd.read_csv(file) for file in kaggle_files]

# df_combined = pd.concat(
#     [pd.read_csv(f).assign(file_name = os.path.basename(f)) for f in kaggle_files], 
#     ignore_index = True
# )

# print(f"Combined {len(kaggle_files)} files into {df_combined.shape[0]} total rows.")

CPU times: user 5 μs, sys: 1 μs, total: 6 μs
Wall time: 9.06 μs
Combined 121 files into 91336 total rows.


In [8]:
df_combined.head()

,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,...,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c,file_name
0,1,0,0.000,0.98,1,-0.265343,0.270043,-0.925565,-0.394123,0.281717,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,trial_lie_001.csv
1,2,0,0.033,0.98,1,-0.269314,0.280146,-0.921405,-0.399485,0.291082,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,trial_lie_001.csv
2,3,0,0.067,0.98,1,-0.276762,0.281786,-0.918694,-0.402257,0.294677,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,trial_lie_001.csv
3,4,0,0.100,0.98,1,-0.267571,0.284353,-0.920624,-0.390941,0.297649,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,trial_lie_001.csv
4,5,0,0.133,0.98,1,-0.251324,0.286230,-0.924613,-0.374932,0.299439,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,trial_lie_001.csv


In [9]:
# # export to pickle in order to not append all CSVs every time
# df_combined.to_pickle('df_kaggle_combined.pkl')

In [4]:
#import pickle

df_combined = pd.read_pickle('df_kaggle_combined.pkl')

# Preparation of Data

In [5]:
#df_combined.columns

# strip whitespace from column names 
df_combined.columns = df_combined.columns.str.strip()

In [6]:
# create label column from file name

conditions = [
    df_combined['file_name'].str.contains('lie', case = False) | 
    df_combined['file_name'].str.contains('deception', case = False),
    
    df_combined['file_name'].str.contains('truth', case = False) |
    df_combined['file_name'].str.contains('true', case = False)
]

choices = ['lie', 'truth']

df_combined['label'] = np.select(conditions, choices, default = pd.NA)

df_combined

,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,...,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c,file_name,label
0,1,0,0.000,0.98,1,-0.265343,0.270043,-0.925565,-0.394123,0.281717,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,trial_lie_001.csv,lie
1,2,0,0.033,0.98,1,-0.269314,0.280146,-0.921405,-0.399485,0.291082,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,trial_lie_001.csv,lie
2,3,0,0.067,0.98,1,-0.276762,0.281786,-0.918694,-0.402257,0.294677,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,trial_lie_001.csv,lie
3,4,0,0.100,0.98,1,-0.267571,0.284353,-0.920624,-0.390941,0.297649,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,trial_lie_001.csv,lie
4,5,0,0.133,0.98,1,-0.251324,0.286230,-0.924613,-0.374932,0.299439,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,trial_lie_001.csv,lie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91331,536,0,19.815,0.98,1,0.399038,0.150437,-0.904509,0.234647,0.184356,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,trial_truth_060.csv,truth
91332,537,0,19.852,0.98,1,0.394478,0.167984,-0.903420,0.232950,0.200715,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,trial_truth_060.csv,truth
91333,538,0,19.889,0.98,1,0.392632,0.175124,-0.902869,0.232472,0.207146,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,trial_truth_060.csv,truth
91334,539,0,19.926,0.98,1,0.402144,0.140985,-0.904657,0.236606,0.176743,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,trial_truth_060.csv,truth


In [7]:
# check that no labels are empty

check_na_label = df_combined[df_combined['label'].isna()]
check_na_label

,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,...,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c,file_name,label


In [9]:
df_combined['label'].value_counts()

label
lie      48873
truth    42463
Name: count, dtype: int64

In [ ]:
df_combined

In [10]:
# keep only lines where confidence >= 0.8 & success == 1

kaggle_dataset = df_combined[(df_combined['confidence'] >= 0.8) & (df_combined['success'] == 1)]
kaggle_dataset

,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,...,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c,file_name,label
0,1,0,0.000,0.98,1,-0.265343,0.270043,-0.925565,-0.394123,0.281717,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,trial_lie_001.csv,lie
1,2,0,0.033,0.98,1,-0.269314,0.280146,-0.921405,-0.399485,0.291082,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,trial_lie_001.csv,lie
2,3,0,0.067,0.98,1,-0.276762,0.281786,-0.918694,-0.402257,0.294677,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,trial_lie_001.csv,lie
3,4,0,0.100,0.98,1,-0.267571,0.284353,-0.920624,-0.390941,0.297649,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,trial_lie_001.csv,lie
4,5,0,0.133,0.98,1,-0.251324,0.286230,-0.924613,-0.374932,0.299439,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,trial_lie_001.csv,lie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91331,536,0,19.815,0.98,1,0.399038,0.150437,-0.904509,0.234647,0.184356,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,trial_truth_060.csv,truth
91332,537,0,19.852,0.98,1,0.394478,0.167984,-0.903420,0.232950,0.200715,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,trial_truth_060.csv,truth
91333,538,0,19.889,0.98,1,0.392632,0.175124,-0.902869,0.232472,0.207146,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,trial_truth_060.csv,truth
91334,539,0,19.926,0.98,1,0.402144,0.140985,-0.904657,0.236606,0.176743,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,trial_truth_060.csv,truth


In [11]:
#kaggle_dataset['frame'].max()

In [12]:
# drop duplicates id any

duplicates = kaggle_dataset.duplicated()
kaggle_dataset[duplicates]

,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,...,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c,file_name,label


In [13]:
kaggle_dataset['video_id'] = kaggle_dataset['file_name'].str.replace('.csv', '', regex = False)

/tmp/ipykernel_109719/1961372867.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kaggle_dataset['video_id'] = kaggle_dataset['file_name'].str.replace('.csv', '', regex = False)


In [14]:
kaggle_dataset

,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,...,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c,file_name,label,video_id
0,1,0,0.000,0.98,1,-0.265343,0.270043,-0.925565,-0.394123,0.281717,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,trial_lie_001.csv,lie,trial_lie_001
1,2,0,0.033,0.98,1,-0.269314,0.280146,-0.921405,-0.399485,0.291082,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,trial_lie_001.csv,lie,trial_lie_001
2,3,0,0.067,0.98,1,-0.276762,0.281786,-0.918694,-0.402257,0.294677,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,trial_lie_001.csv,lie,trial_lie_001
3,4,0,0.100,0.98,1,-0.267571,0.284353,-0.920624,-0.390941,0.297649,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,trial_lie_001.csv,lie,trial_lie_001
4,5,0,0.133,0.98,1,-0.251324,0.286230,-0.924613,-0.374932,0.299439,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,trial_lie_001.csv,lie,trial_lie_001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91331,536,0,19.815,0.98,1,0.399038,0.150437,-0.904509,0.234647,0.184356,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,trial_truth_060.csv,truth,trial_truth_060
91332,537,0,19.852,0.98,1,0.394478,0.167984,-0.903420,0.232950,0.200715,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,trial_truth_060.csv,truth,trial_truth_060
91333,538,0,19.889,0.98,1,0.392632,0.175124,-0.902869,0.232472,0.207146,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,trial_truth_060.csv,truth,trial_truth_060
91334,539,0,19.926,0.98,1,0.402144,0.140985,-0.904657,0.236606,0.176743,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,trial_truth_060.csv,truth,trial_truth_060


# Export Final Dataset

In [15]:
kaggle_dataset.to_pickle('kaggle_openface_merged.pkl')
kaggle_dataset.to_csv('kaggle_openface_merged.csv', index = False)